# AA Tau

### Quick Links

* Data Reduction
    - Optical
        - [Optical 2014](#opt14)
           - [Dec 12](#opt14_12)
           - [Dec 02](#opt14_02)
        - [Optical 2008](#opt08)
    - [Infrared](#ir)
* [Combine opt and ir](#scaling)
    - [Photometry](#photometry)
        - [Bouvier/Schneider photometry](#googledocs)
* [Extinction](#ext)
    - [Fits](#fit)
    - [Grey extinction?](#grey)

[Write data to files](#write)

[Figures for the paper](#figures)

In [1]:
import numpy as np
import tarfile

In [2]:
import matplotlib.pyplot as plt
# %matplotlib notebook  
%matplotlib inline 

In [3]:
from astropy.io import fits

In [4]:
import scipy.interpolate as interp
from sys import platform

# Optical 2014 <a name="opt14"></a>



> From: Gregory Herczeg <gherczeg1@gmail.com>  
Date: Mon, Sep 21, 2015 at 6:38 PM  
Subject: AA Tau spectra  
To: Kevin Covey <kevin.covey@wwu.edu>  

> ...Most of this data was obtained at UH88, the spectra are from the automated reduction.  I've developed an alternative pipeline for the red, but not the blue, and without fluxcal.  So, for now, this is what we have.  We also have V-band from acquisition images, so it should be possible to test the fluxcal that way.

> The one ascii file is from DBSP in 2008 and should be good, with accurate fluxcal.

> ftp://ftp.mpe.mpg.de/people/gregoryh/forkc


From Kevin:

> While documenting everything for an 'observations' section, I was able to figure out the timing of the optical spectra that were observed in 2014.  It looks like spec_C14_346_066_001_17_R.fits and spec_C14_346_199_001_17_R.fits (and B.fits) are the closest optical spectra to our Dec. 12 NIR observations, and spec_C14_336_061_001_17_R.fits and spec_C14_336_162_001_17_R.fits (and B.fits, of course) are closest to our Dec. 2nd NIR spectra. 

Dec 12, 2014

spec_C14_346_066_001_17_R.fits and B.fits (later found to be bad in B, see below) 

spec_C14_346_199_001_17_R.fits and B.fits


Dec 2, 2014

spec_C14_336_061_001_17_R.fits and B.fits

spec_C14_336_162_001_17_R.fits and B.fits


In [5]:
# aatautar = tarfile.open('../../DATA/AATau.tar.gz')
# aatautar.list()
# aatautar.extractall(path="../../DATA/")
# aatautar.close()

## Dec 12, 2014 <a name="opt14_12"></a>

**R band first:**

In [6]:
hdulist = fits.open('../..//DATA/AATau/Optical/spec_C14_346_066_001_17_R.fits')
# hdulist.info()
Rdata=hdulist[0].data
Rstart=hdulist[0].header['CRVAL1']
Rstep=hdulist[0].header['CDELT1']
Rwavelength_346_066=np.arange(Rstart,Rstart+Rstep*Rdata.size,Rstep)*1.0E-4 # microns
Rdata_346_066=hdulist[0].data
hdulist.close()

FileNotFoundError: [Errno 2] No such file or directory: '../..//DATA/AATau/Optical/spec_C14_346_066_001_17_R.fits'

In [ ]:
hdulist = fits.open('../../DATA/AATau/Optical/spec_C14_346_199_001_17_R.fits')
# hdulist.info()
Rdata=hdulist[0].data
Rstart=hdulist[0].header['CRVAL1']
Rstep=hdulist[0].header['CDELT1']
Rwavelength_346_199=np.arange(Rstart,Rstart+Rstep*Rdata.size,Rstep)*1.0E-4 # MICRONS MICRONS MICRONS!
Rdata_346_199=hdulist[0].data
hdulist.close()

In [ ]:
hdulist[0].header['FLXUNITS']

In [ ]:
fig,ax=plt.subplots()
ax.plot(Rwavelength_346_066,Rdata_346_066,label="066")
ax.plot(Rwavelength_346_199,Rdata_346_199,label="199")
ax.set_title("AA Tau: R band 12Dec14")
ax.legend(loc="best")
plt.show()

In [ ]:
np.min(Rwavelength_346_199),np.min(Rwavelength_346_066)  # They match

Step 1 = Shift horizontally by matching Halpha peaks, scale vertically by matching the average flux in the flats from 0.78 to 0.92 microns

In [ ]:
shift=np.argmax(Rdata_346_066)-np.argmax(Rdata_346_199)
shift

In [ ]:
Rwavelength_346_066[9]-Rwavelength_346_066[0]

In [ ]:
scale = np.average(Rdata_346_066[np.logical_and(Rwavelength_346_066 < 0.92, 
                                                Rwavelength_346_066 > 0.78)]) \
    / np.average(Rdata_346_199[np.logical_and(Rwavelength_346_199 < 0.92, 
                                              Rwavelength_346_199 > 0.78)]) 
scale    

The problem might be that the wavelengths aren't true without the horizontal shift, but we'll ignore that for now.

Use the 199 wavelengths: chop off the end and shift+scale the 066 onto it

In [ ]:
fig,ax=plt.subplots()
ax.plot(Rwavelength_346_066[shift:], Rdata_346_066[shift:]/scale,label="066 scaled")
ax.plot(Rwavelength_346_066[shift:], Rdata_346_199[:-shift],label="199")

Rdata_dec12=(Rdata_346_199[:-shift]+Rdata_346_066[shift:]/scale)/2.0
Rwavelength_dec12=Rwavelength_346_066[shift:]
ax.plot(Rwavelength_dec12,Rdata_dec12,label="average")
ax.set_title("AA Tau: R band 12Dec14")
ax.legend(loc="best")
plt.show()

**Next B:**

In [ ]:
hdulist = fits.open('../../DATA/AATau/Optical/spec_C14_346_066_001_17_B.fits')
# hdulist.info()
Bdata=hdulist[0].data
Bstart=hdulist[0].header['CRVAL1']
Bstep=hdulist[0].header['CDELT1']
Bwavelength_346_066=np.arange(Bstart,Bstart+Bstep*Bdata.size,Bstep)*1.0E-4 # microns
Bdata_346_066=hdulist[0].data
hdulist.close()

hdulist = fits.open('../../DATA/AATau/Optical/spec_C14_346_199_001_17_B.fits')
# hdulist.info()
Bdata=hdulist[0].data
Bstart=hdulist[0].header['CRVAL1']
Bstep=hdulist[0].header['CDELT1']
Bwavelength_346_199=np.arange(Bstart,Bstart+Bstep*Bdata.size,Bstep)*1.0E-4 # microns
Bdata_346_199=hdulist[0].data
hdulist.close()


Yikes!  346_066 is way too messy in B:

In [ ]:
fig,ax=plt.subplots()
ax.plot(Bwavelength_346_066,Bdata_346_066,label="066")
ax.plot(Bwavelength_346_199,Bdata_346_199,label="199")
ax.set_title("AA Tau: B band 12Dec14")
ax.legend(loc="best")
plt.show()

Use only the 199 data (no shift, no scale):

In [ ]:
fig,ax=plt.subplots()
Bdata_dec12=Bdata_346_199
Bwavelength_dec12=Bwavelength_346_199
ax.plot(Bwavelength_dec12,Bdata_dec12,label="199 only")
ax.set_title("AA Tau: B band 12Dec14")
ax.legend(loc="best")
plt.show()

**Match R and B?  Bottom line is that they don't overlap that much**

In [ ]:
fig,ax=plt.subplots()
ax.plot(Bwavelength_dec12,Bdata_dec12)
ax.plot(Rwavelength_dec12[:100],Rdata_dec12[:100])
ax.set_title("AA Tau: B and part of R 12Dec14")
plt.show()

In [ ]:
Bwavelength_dec12[-20:],Rwavelength_dec12[:20]

Conclusion: lop off points 0-7 of R band to "match"

In [ ]:
wavelength_dec12=np.append(Bwavelength_dec12,Rwavelength_dec12[8:])
data_dec12=np.append(Bdata_dec12,Rdata_dec12[8:])
fig,ax=plt.subplots()
ax.plot(wavelength_dec12,data_dec12)
ax.set_title("AA Tau: 12Dec14")
plt.show()

## Dec 2, 2014 <a name="opt14_02"></a>

**R and B, just as above:**

In [ ]:
hdulist = fits.open('../../DATA/AATau/Optical/spec_C14_336_061_001_17_R.fits')
# hdulist.info()
Rdata=hdulist[0].data
Rstart=hdulist[0].header['CRVAL1']
Rstep=hdulist[0].header['CDELT1']
Rwavelength_336_061=np.arange(Rstart,Rstart+Rstep*Rdata.size,Rstep)*1.0E-4 # microns
Rdata_336_061=hdulist[0].data
hdulist.close()

hdulist = fits.open('../../DATA/AATau/Optical/spec_C14_336_162_001_17_R.fits')
# hdulist.info()
Rdata=hdulist[0].data
Rstart=hdulist[0].header['CRVAL1']
Rstep=hdulist[0].header['CDELT1']
Rwavelength_336_162=np.arange(Rstart,Rstart+Rstep*Rdata.size,Rstep)*1.0E-4 # microns
Rdata_336_162=hdulist[0].data
hdulist.close()

shift=np.argmax(Rdata_336_061)-np.argmax(Rdata_336_162)

scale = np.average(Rdata_336_061[np.logical_and(Rwavelength_336_061 < 0.92, 
                                                Rwavelength_336_061 > 0.78)]) \
    / np.average(Rdata_336_162[np.logical_and(Rwavelength_336_162 < 0.92, 
                                              Rwavelength_336_162 > 0.78)]) 
shift,scale    


In [ ]:
hdulist[0].header['FLXUNITS']

In [ ]:
fig,ax=plt.subplots()
ax.plot(Rwavelength_336_061[shift:],Rdata_336_061[shift:])
ax.plot(Rwavelength_336_162[shift:],scale * Rdata_336_162[:-shift])

Rdata_dec02=(scale*Rdata_336_162[:-shift]+Rdata_336_061[shift:])/2.0
Rwavelength_dec02=Rwavelength_336_061[shift:]
ax.plot(Rwavelength_dec02,Rdata_dec02)
ax.set_title("AA Tau: 02Dec14")

plt.show()

In [ ]:
hdulist = fits.open('../../DATA/AATau/Optical/spec_C14_336_061_001_17_B.fits')
# hdulist.info()
Bdata=hdulist[0].data
Bstart=hdulist[0].header['CRVAL1']
Bstep=hdulist[0].header['CDELT1']
Bwavelength_336_061=np.arange(Bstart,Bstart+Bstep*Bdata.size,Bstep)*1.0E-4 # microns
Bdata_336_061=hdulist[0].data
hdulist.close()

hdulist = fits.open('../../DATA/AATau/Optical/spec_C14_336_162_001_17_B.fits')
# hdulist.info()
Bdata=hdulist[0].data
Bstart=hdulist[0].header['CRVAL1']
Bstep=hdulist[0].header['CDELT1']
Bwavelength_336_162=np.arange(Bstart,Bstart+Bstep*Bdata.size,Bstep)*1.0E-4 # microns
Bdata_336_162=hdulist[0].data
hdulist.close()


In [ ]:
fig,ax=plt.subplots()
ax.plot(Bwavelength_336_061,Bdata_336_061)
ax.plot(Bwavelength_336_162,Bdata_336_162)

# These look good, so just average:

Bdata_dec02=(Bdata_336_162+Bdata_336_061)/2.0
Bwavelength_dec02=Bwavelength_336_061
ax.plot(Bwavelength_dec02,Bdata_dec02)
ax.set_title("AA Tau: 02Dec14")
plt.show()

**Do R and B match?  Better than Dec 12!**

In [ ]:
fig,ax=plt.subplots()
ax.plot(Bwavelength_dec02,Bdata_dec02)
ax.plot(Rwavelength_dec02[:100],Rdata_dec02[:100])
ax.set_title("AA Tau: 02Dec14")
plt.show()

In [ ]:
Bwavelength_dec02[-20:],Rwavelength_dec02[:20]

Conclusion: To match, lop points 0-12 off of R to "match"

In [ ]:
wavelength_dec02=np.append(Bwavelength_dec02,Rwavelength_dec02[13:])
data_dec02=np.append(Bdata_dec02,Rdata_dec02[13:])

fig,ax=plt.subplots()
ax.plot(wavelength_dec02,data_dec02)
ax.set_title("AA Tau: 02Dec14")
plt.show()

# Optical 2008 <a name="opt08"></a>

In [ ]:
wavelength2, data2, junk1, junk2 = np.loadtxt('../../DATA/AATau/Optical/aatau.dat', unpack=True,skiprows=9)
wavelength2=wavelength2/10000.0
wavelength3, data3, junk3, junk4 = np.loadtxt('../../DATA/AATau/Optical/aataublue.dat', unpack=True,skiprows=9)
wavelength3=wavelength3/10000.0
wavelength_2008=np.append(wavelength3[::-1],wavelength2)
data_2008=np.append(data3[::-1],data2)



In [ ]:
fig,ax=plt.subplots()
ax.plot(wavelength_dec02,data_dec02,label='dec 02 2014')
ax.plot(wavelength_dec12,data_dec12,label='dec 12 2014')
ax.plot(wavelength_2008,data_2008,label='2008 (fluxcal)')
ax.legend(loc='best')
ax.set_ylim(0,0.7E-13)
ax.set_xlabel("wavelength (microns)")
ax.set_ylabel("Flux (erg/s/cm2/A)")
ax.set_title("Optical Spectra")
plt.show()

Wavelength looks good, features in the same place.

# Infrared <a name="ir"></a>

-- 2014 --  
Dec 2:

merge.AA_Tau.3-10.fits  
merge.AA_Tau.28-31.fits  
merge.AA_Tau.84-87.fits  

Dec 12:

comb.AA_Tau.31-34.fits  
comb.AA_Tau.59-62.fits  
comb.AA_Tau.111-114.fits  

-- 2008 --  
final.AATau.31-34.fits

Kevin rewrote the files as AA_Tau.Dec2_2014.3-10.txt, etc:

In [ ]:
IRwav2a, IRdata2a, junk= np.loadtxt('../../DATA/AATau/IR/AA_Tau.Dec2_2014.3-10.txt', unpack=True)
IRwav2b, IRdata2b, junk= np.loadtxt('../../DATA/AATau/IR/AA_Tau.Dec2_2014.28-31.txt', unpack=True)
IRwav2c, IRdata2c, junk= np.loadtxt('../../DATA/AATau/IR/AA_Tau.Dec2_2014.84-87.txt', unpack=True)
IRwav12a, IRdata12a, junk= np.loadtxt('../../DATA/AATau/IR/AA_Tau.Dec12_2014.31-34.txt', unpack=True)
IRwav12b, IRdata12b, junk= np.loadtxt('../../DATA/AATau/IR/AA_Tau.Dec12_2014.59-62.txt', unpack=True)
IRwav12c, IRdata12c, junk= np.loadtxt('../../DATA/AATau/IR/AA_Tau.Dec12_2014.111-114.txt', unpack=True)


hdulist = fits.open('../../DATA/AATau/IR/final.AATau.31-34.fits')
IRwavelength_2008=hdulist[0].data[0]
IRdata_2008=hdulist[0].data[1]
hdulist.close()


All IR wavelengths have a hitch around element 5000??

2a and 2c are 7099 elements, 2b is 7100 elements

12a is 7063, 12b is 7080, 12c is 7088

Bottom line is that they are all different!  

In [ ]:
# mask out the NaNs:
def maskNans(wavelength,data):
    
    masked_data=data[np.logical_not(np.isnan(data))]
    masked_wavelength=wavelength[np.logical_not(np.isnan(data))]

    return (masked_wavelength,masked_data)

IRwav2a, IRdata2a = maskNans(IRwav2a, IRdata2a)
IRwav2b, IRdata2b = maskNans(IRwav2b, IRdata2b)
IRwav2c, IRdata2c = maskNans(IRwav2c, IRdata2c)
IRwav12a, IRdata12a = maskNans(IRwav12a, IRdata12a)
IRwav12b, IRdata12b = maskNans(IRwav12b, IRdata12b)
IRwav12c, IRdata12c = maskNans(IRwav12c, IRdata12c)

IRwavelength_2008, IRdata_2008 = maskNans(IRwavelength_2008, IRdata_2008)


In [ ]:
fig,[ax1,ax2]=plt.subplots(1,2)
ax1.plot(IRwav2a,IRdata2a,label='2014Dec02a')
ax1.plot(IRwav2b,IRdata2b,label='2014Dec02b')
ax1.plot(IRwav2c,IRdata2c,label='2014Dec02c')
ax2.plot(IRwav12a,IRdata12a,label='2014Dec12a')
ax2.plot(IRwav12b,IRdata12b,label='2014Dec12b')
ax2.plot(IRwav12c,IRdata12c,label='2014Dec12c')
ax1.legend(loc='best')
ax1.set_ylim(0,0.6e-13)
ax2.legend(loc='best')
ax2.set_ylim(0,0.6e-13)
plt.show()

Wavelength calibration looks good.  The third on Dec 2 and the second on Dec 12 do not look like the other two.  Why?

# Scaling IR and Optical <a name="scaling"></a>

In [ ]:
fig,ax=plt.subplots()
ax.plot(IRwavelength_2008,IRdata_2008)
ax.plot(wavelength_2008,data_2008)
ax.set_ylim(0,1E-13)
ax.set_title("AA Tau: all 2008")
plt.show()

The 2008 infrared matches reasonably with the 2008 optical, which is flux calibrated.  Therefore, don't do any scaling on the 2008.

There are three scaling tasks for the 2014 data: 
* scale the three infrared to match each other
* scale the infrared to the optical
* scale the overall 2014 spectrum by some unknown amount.  

Getting the opt-ir to fit for the 2014 is easy because they overlap:

In [ ]:
fig,ax=plt.subplots()
ax.plot(IRwav2a,IRdata2a,label='2014Dec02a')
ax.plot(IRwav2b,IRdata2b,label='2014Dec02b')
ax.plot(IRwav2c,IRdata2c,label='2014Dec02c')
ax.plot(wavelength_dec02,data_dec02,'k',label='2014 optical')
ax.plot(IRwav2a[500:2400],IRdata2a[500:2400],'purple',label='normalize here')
ax.legend(loc='best')
ax.set_title("AA Tau: all 2014 Dec 02")
ax.set_ylim(-0.05E-13,0.5E-13)
plt.show()

In [ ]:
IRdata2b_scale=IRdata2b*np.mean(IRdata2a[500:2400])/np.mean(IRdata2b[500:2400])
IRdata2c_scale=IRdata2c*np.mean(IRdata2a[500:2400])/np.mean(IRdata2c[500:2400])

fig,ax=plt.subplots()
ax.plot(IRwav2a,IRdata2a,label='2014Dec02a')
ax.plot(IRwav2b,IRdata2b_scale,label='2014Dec02b')
ax.plot(IRwav2c,IRdata2c_scale,label='2014Dec02c')
ax.plot(wavelength_dec02,data_dec02,'k',label='2014 optical')
ax.plot(IRwav2a[500:2400],IRdata2a[500:2400],'purple',label='normalize here')
ax.legend(loc='best')
ax.set_title("AA Tau: all 2014 Dec 02")
ax.set_ylim(-0.05E-13,0.5E-13)
plt.show()


Conclusion:  The third spectrum does not seem to go, because a simple scaling won't bring the long wavelength in line.  Keep the first two, scale to fit in the purple region, then average.

In [ ]:
np.mean(IRdata2b[500:2400])/np.mean(IRdata2a[500:2400])


In [ ]:
IRwav2a[500],IRwav2a[2400],IRwav2b[500],IRwav2b[2400],IRwav2c[500],IRwav2c[2400]


In [ ]:
f_IRdat2b=interp.interp1d(IRwav2b,IRdata2b_scale)

IRwavelength_dec02=IRwav2a[:-5]
IRdata_dec02=(IRdata2a[:-5]+f_IRdat2b(IRwavelength_dec02))/2.0

fig,ax=plt.subplots()
ax.plot(IRwavelength_dec02,IRdata_dec02,label='2014Dec02')
ax.plot(wavelength_dec02,data_dec02,'k',label='2014 optical')
ax.set_ylim(-0.05E-13,0.5E-13)
ax.set_xlim(0.8,1.1)
ax.set_title("AA Tau: all 2014 Dec 02")
plt.show()


Not sure about the hitch in the long end of the optical... Match
the slope in 0.85 to 0.90 microns projected to the value at 0.945?  Yes, this works.

In [ ]:
fitrange=np.logical_and(wavelength_dec02>0.85,wavelength_dec02<0.90)
m,b=np.polyfit(wavelength_dec02[fitrange],data_dec02[fitrange],1)
m*0.945+b

In [ ]:
IRrange=np.logical_and(IRwavelength_dec02>0.94,IRwavelength_dec02<0.95)
np.average(IRdata_dec02[IRrange])

In [ ]:
(m*0.945+b)/(np.average(IRdata_dec02[IRrange]))

So, ~~multiply the IR~~ divide the optical by 1.0071

In [ ]:

IRwavelength_dec02=IRwav2a[:-5]
IRdata_dec02=(IRdata2a[:-5]+f_IRdat2b(IRwavelength_dec02))/2.0
data_dec02scale=data_dec02/1.0071

fig,ax=plt.subplots()
ax.plot(IRwavelength_dec02,IRdata_dec02,label='2014Dec02')
ax.plot(wavelength_dec02,data_dec02scale,'k',label='2014 optical')
ax.set_ylim(-0.05E-13,0.5E-13)
ax.set_xlim(0.8,1.1)
ax.set_title("AA Tau: all 2014 Dec 02")
plt.show()


**Repeat for the other night**

In [ ]:
IRdata12b_scale=IRdata12b*np.mean(IRdata12a[500:2400])/np.mean(IRdata12b[500:2400])
IRdata12c_scale=IRdata12c*np.mean(IRdata12a[500:2400])/np.mean(IRdata12c[500:2400])

fig,ax=plt.subplots()
ax.plot(IRwav12a,IRdata12a,label='2014Dec12a')
ax.plot(IRwav12b,IRdata12b_scale,label='2014Dec12b')
ax.plot(IRwav12c,IRdata12c_scale,label='2014Dec12c')
ax.plot(wavelength_dec02,data_dec02,'k',label='2014 optical')
ax.plot(IRwav12a[500:2400],IRdata12a[500:2400],'purple',label='normalize here')
ax.legend(loc='best')
ax.set_ylim(-0.05E-13,0.5E-13)
ax.set_title("AA Tau: all 2014 Dec 12")
plt.show()


These look better.

In [ ]:
np.mean(IRdata12a[500:2400])/np.mean(IRdata12b[500:2400]),np.mean(IRdata12a[500:2400])/np.mean(IRdata12c[500:2400])

In [ ]:
f_IRdat12b=interp.interp1d(IRwav12b,IRdata12b_scale)
f_IRdat12c=interp.interp1d(IRwav12c,IRdata12c_scale)

IRwavelength_dec12=IRwav12a
IRdata_dec12=(IRdata12a+f_IRdat12b(IRwavelength_dec12)+f_IRdat12b(IRwavelength_dec12))/3.0

fig,ax=plt.subplots()
ax.plot(IRwavelength_dec12,IRdata_dec12,label='2014Dec12')
ax.plot(wavelength_dec12,data_dec12,'k',label='2014 optical')
ax.set_ylim(-0.05E-13,0.5E-13)
ax.set_xlim(0.8,1.1)
ax.set_title("AA Tau: all 2014 Dec 12")
plt.show()


In [ ]:
fitrange=np.logical_and(wavelength_dec12>0.85,wavelength_dec12<0.90)
m,b=np.polyfit(wavelength_dec12[fitrange],data_dec12[fitrange],1)
m*0.945+b

In [ ]:
IRrange=np.logical_and(IRwavelength_dec12>0.94,IRwavelength_dec12<0.95)
np.average(IRdata_dec12[IRrange])
# IRdata_dec12[IRrange]

In [ ]:
(m*0.945+b)/(np.average(IRdata_dec12[IRrange]))

So, ~~multiply the IR~~ divide the optical by 1.5214 (multiply by 0.657) to match. 

In [ ]:
f_IRdat12b=interp.interp1d(IRwav12b,IRdata12b_scale)
f_IRdat12c=interp.interp1d(IRwav12c,IRdata12c_scale)

IRwavelength_dec12=IRwav12a
IRdata_dec12=(IRdata12a+f_IRdat12b(IRwavelength_dec12)+f_IRdat12b(IRwavelength_dec12))/3.0
data_dec12scale=data_dec12/1.5214

fig,ax=plt.subplots()
ax.plot(wavelength_dec12,data_dec12scale,'k',label='2014 optical')
ax.plot(IRwavelength_dec12,IRdata_dec12,label='2014Dec12')
ax.set_ylim(-0.05E-13,0.5E-13)
ax.set_xlim(0.8,1.1)
ax.set_title("AA Tau: all 2014 Dec 12")
plt.show()

In [ ]:
fig,ax=plt.subplots()
ax.plot(IRwavelength_2008,IRdata_2008,'r')
ax.plot(wavelength_2008,data_2008,'r',label='2008')
ax.plot(IRwavelength_dec02,IRdata_dec02,'b',label='2014Dec02')
ax.plot(wavelength_dec02,data_dec02scale,'b')
ax.plot(IRwavelength_dec12,IRdata_dec12,'k',label='2014Dec12')
ax.plot(wavelength_dec12,data_dec12scale,'k')
ax.legend(loc='best')
ax.set_title('AA Tau')
ax.set_ylim(-0.05E-13,0.7E-13)
plt.show()

To recap, there are three scaling tasks for the 2014 data: 
* ~~scale the three infrared to match each other~~
* ~~scale the infrared to the optical~~
* scale the overall 2014 spectrum by some unknown amount.  


### To do that, we need photometry.  To do photometry, we need units.

### Units:  1 erg/s = 1E-7 W, so 1 erg/s cm^2 = 1E-3 W/ m^2 and 1 erg/s cm^2 A = 10 W/ m^2 um

In [ ]:
from astropy import units as u

In [ ]:
IRdata_dec02u = IRdata_dec02    * u.erg / (u.s * u.cm * u.cm * u.AA)
IRdata_dec12u = IRdata_dec12    * u.erg / (u.s * u.cm * u.cm * u.AA)
IRdata_2008u  = IRdata_2008     * u.erg / (u.s * u.cm * u.cm * u.AA)
data_dec02u   = data_dec02scale * u.erg / (u.s * u.cm * u.cm * u.AA)
data_dec12u   = data_dec12scale * u.erg / (u.s * u.cm * u.cm * u.AA)
data_2008u    = data_2008       * u.erg / (u.s * u.cm * u.cm * u.AA)


In [ ]:
IRwavelength_dec02u = IRwavelength_dec02 * u.micron
IRwavelength_dec12u = IRwavelength_dec12 * u.micron
IRwavelength_2008u  =  IRwavelength_2008 * u.micron
wavelength_dec02u   =   wavelength_dec02 * u.micron
wavelength_dec12u   =   wavelength_dec12 * u.micron
wavelength_2008u    =    wavelength_2008 * u.micron

# Use photometry to constrain the scaling: <a name="photometry"></a>

Options:

A. Tie the whole scale to the AAVSO data:  V(2014) = 15.0

B. Use magnitudes from the literature:

> <em>X-ray to NIR emission from AA Tauri during the dim state - Occultation of the inner disk and gas-to-dust ratio of the absorber</em>

> P. C. Schneider, K. France, H. M. Günther, G. J. Herczeg, J. Robrade, J. Bouvier, M. McJunkin, J. H. M. M. Schmitt

> (Submitted on 16 Sep 2015)

These new observations happen just at the end of their plot, so 15.2 ish.  


C. **From Kevin: V = 12.73$\pm$0.35 in 2008 to V = 14.95$\pm$0.43 in 2014** (dated 2/22/18 from figure)

**So, accept all scaling that will yield between V = 14.52 and V = 15.38 in 2014.**

In [ ]:
if platform=='win32':
    %run utils\getMag.py
else:  
    %run utils/getMag.py

No scaling:

In [ ]:
getMag('V',wavelength_2008u,data_2008u)

In [ ]:
getMag('V',wavelength_dec02u,data_dec02u)

In [ ]:
getMag('V',wavelength_dec12u,data_dec12u)

Hunt and peck scaling here: 

Now require each spectrum gets fainter in K?  Check Ks:

In [ ]:
# NO LONGER NEEDED, KEVIN TOOK CARE OF THE NaNs
# mask out the NaNs:
def maskNans(wavelength,data):
    masked_data=data[np.logical_not(np.isnan(data))]
    masked_wavelength=wavelength[np.logical_not(np.isnan(data))]
    return (masked_wavelength,masked_data)

In [ ]:
getMag('Ks',IRwavelength_2008u,IRdata_2008u)

In [ ]:
getMag('Ks',IRwavelength_dec02u,IRdata_dec02u)

In [ ]:
getMag('Ks',IRwavelength_dec12u,IRdata_dec12u)

## Scaling, Revisited <a name="googledocs"></a>

Now use the Bouvier/Schneider photometry on google docs.  Consider everything at the fall 2011 campaign and more recent.

 | V | R | J | H | K
--- | --- | --- | --- | ---
min | 13.50 | 12.04 | 10.46 | 9.20 | 8.43
max | 16.49 | 14.83  | 11.13 | 10.13 | 9.70

In [ ]:
test=np.array([])

for j in np.arange(0.2,2.0,.01):
    if 9.70 > getMag('Ks',IRwavelength_dec02u,IRdata_dec02u*j) > 8.43:
        test=np.append(test,j)
        
print,'min=',np.min(test),' max=',np.max(test)

In [ ]:
for j in np.arange(0.2,2.0,.01):
    if 10.13 > getMag('H',IRwavelength_dec02u,IRdata_dec02u*j) > 9.2:
        test=np.append(test,j)
        
print,'min=',np.min(test),' max=',np.max(test)

In [ ]:
for j in np.arange(0.2,2.0,.01):
    if 11.13 > getMag('J',IRwavelength_dec02u,IRdata_dec02u*j) > 10.46:
        test=np.append(test,j)
  
print,'min=',np.min(test),' max=',np.max(test)

These NIR magnitudes are very faint, and the optical magnitudes are over too large a range.  Look what scaling like this does to the optical mags:

In [ ]:
getMag('V',wavelength_dec02u,data_dec02u*0.26),getMag('V',wavelength_dec02u,data_dec02u*0.80)

So, the V = 15.2 is just barely consistent with Kevin's read of the AAVSO data.  

My read of the AAVSO data is that V could be anywhere between 14.3 and 15.8:

In [ ]:
test=np.array([])
for j in np.arange(0.2,2.0,.01):
    if 15.8 > getMag('V',wavelength_dec02u,data_dec02u*j) > 14.3:
        test=np.append(test,j)
print,'min=',np.min(test),' max=',np.max(test)

The V=13.5 and V=19.5 from the google docs is just too big a range to be useful.

In [ ]:
test=np.array([])
for j in np.arange(0.2,2.0,.01):
    if 14.83 > getMag('R',wavelength_dec02u,data_dec02u*j) > 12.04:
        test=np.append(test,j)
print,'min=',np.min(test),' max=',np.max(test)

In [ ]:
# Repeat for the other night:
test=np.array([])
for j in np.arange(0.2,2.0,.01):
    if 9.7 > getMag('Ks',IRwavelength_dec12u,IRdata_dec12u*j) > 8.43:
        test=np.append(test,j)
print,'min=',np.min(test),' max=',np.max(test)

In [ ]:
# Repeat for the other night:
test=np.array([])
for j in np.arange(0.2,2.0,.01):
    if 15.8 > getMag('V',wavelength_dec12u,data_dec12u*j) > 14.3:
        test=np.append(test,j)
print,'min=',np.min(test),' max=',np.max(test)

Conclusion: Scale Dec02 by 0.46 -> 1.82 and Dec12 by 0.50 -> 1.97 based on AAVSO V only.

# Plot extinction curve: <a name="ext"></a>


In [ ]:
allwav_2008,alldata_2008=np.append(wavelength_2008,IRwavelength_2008),np.append(data_2008,IRdata_2008)
allwav_dec02,alldata_dec02=np.append(wavelength_dec02,IRwavelength_dec02),np.append(data_dec02scale,IRdata_dec02)
allwav_dec12,alldata_dec12=np.append(wavelength_dec12,IRwavelength_dec12),np.append(data_dec12scale,IRdata_dec12)

allwav_2008u = allwav_2008*wavelength_2008u.unit
alldata_2008u = alldata_2008*data_2008u.unit
allwav_dec02u = allwav_dec02*wavelength_dec02u.unit
alldata_dec02u = alldata_dec02*data_dec02u.unit
allwav_dec12u = allwav_dec12*wavelength_dec12u.unit
alldata_dec12u = alldata_dec12*data_dec12u.unit

In [ ]:
interp2008=interp.interp1d(allwav_2008,alldata_2008)

ext_dec02=-2.5*np.log10(alldata_dec02/interp2008(allwav_dec02))

fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))
ax1.plot(allwav_dec02,ext_dec02,'k')
ax2.plot(1./allwav_dec02,ext_dec02,'k')

ax1.set_xlabel('$\lambda$ [micron]')
ax2.set_xlabel('$1/\lambda$ [1/micron]')
ax1.set_ylabel('$A_\lambda$')

ax1.set_title('AA Tau Extinction Event')
ax2.set_title('AA Tau Extinction Event')
ax1.set_ylim(-0.5,4)
ax2.set_ylim(-0.5,4)
plt.show()

In [ ]:
if platform=='win32':
    %run utils\getFMext
else:  
    %run utils/getFMext.py

In [ ]:
fig,(ax1,ax2)=plt.subplots(2,1,figsize=(10,10),gridspec_kw = {'height_ratios':[3, 1]})

# fig.subplots_adjust(hspace=0)

# my little FMunred doesn't include UV, lambda < 0.411 micron

ax1.plot(allwav_2008,alldata_2008,'grey',label='2008 observed')
ax1.plot(allwav_dec02,alldata_dec02,'k',label='02dec2014 observed')
ax1.plot(allwav_2008[allwav_2008>0.411],alldata_2008[allwav_2008>0.411]*
        10.0**(-0.4*2.0*getFMext(allwav_2008[allwav_2008>0.411],3.0,'fmunred')/3.0),'blue',label="2008 ext'd AV=2.0 RV=3")
ax1.set_ylim(0,0.8E-13)
ax1.set_xlim(0.25,2.5)
ax1.set_xlabel('$\lambda$ [micron]')
ax1.set_ylabel('$F_\lambda$ (erg s$^{-1}$ cm$^{-2}$ $\AA^{-1}$)')
ax1.legend(loc='best')
ax1.set_title('AA Tau')

ax2.plot([.25,2.5],[0,0],'k')
ax2.plot(allwav_dec02[allwav_dec02>0.411],
         interp2008(allwav_dec02[allwav_dec02>0.411])*10.0**(-0.4*2.0*getFMext(allwav_dec02[allwav_dec02>0.411],3.0,'fmunred')/3.0)
         -alldata_dec02[allwav_dec02>0.411],'blue',label='Extinction only')
ax2.set_xlim(0.25,2.5)
ax2.set_ylim(-1E-14,1E-14)
ax2.set_xlabel('$\lambda$ [micron]')
ax2.set_ylabel('Model $-$ Observed (erg s$^{-1}$ cm$^{-2}$ $\AA^{-1}$)')

plt.show()

In [ ]:
IRwavelength_dec02.max(),IRwavelength_dec12.max(),allwav_2008.max()

## Fit to NIR extinction curve: <a name="fit"></a>

Quick-and-dirty remove the telluric lines:

In [ ]:
IRext_dec02=-2.5*np.log10(IRdata_dec02/interp2008(IRwavelength_dec02))

notel_ext=IRext_dec02[IRext_dec02 > -0.1]
notel_wav=IRwavelength_dec02[IRext_dec02 > -0.1]

fig,ax=plt.subplots(1,1)
ax.plot(1/allwav_dec02,ext_dec02)
ax.plot(1/notel_wav,notel_ext)
ax.set_xlabel('inverse wavelength')
ax.set_ylabel('extinction')
fig.show()

$A_\lambda/A_V$ from the literature:

FM_unred (F99) = cubic spline, explicitly $R_V$ independent 

FM07 $= (0.63 - 0.84/R_V)\lambda^{-1.84}$

FM09 $= (0.349/R_V + 2.087) / (1 + (\lambda/0.507)^\alpha)$, where $\alpha$ is in the range from 1.5 to 3.0

In [ ]:
fig,ax=plt.subplots(1,1)
ax.plot(1/notel_wav,notel_ext,'black',alpha=0.5)

for A in [2,3]:
    ax.plot(1/notel_wav,A*getFMext(notel_wav,3.0,'fmunred')/3.0,'black',label='F99 Av='+str(A))
    for R in [2,4]:
        ax.plot(1/notel_wav,A*(0.65-0.84/R)*notel_wav**-1.84,'blue',label='FM07 Rv='+str(R)+' Av='+str(A))
        ax.plot(1/notel_wav,A*(0.349/R+2.087)/(1+(notel_wav/0.507)**2.5),'green',label='FM09 Rv='+str(R)+' Av='+str(A))
    
ax.set_xlabel('$\lambda^{-1}$ [$\mu$m$^{-1}$]')
ax.set_ylabel('$A_\lambda$  (observed)')
ax.legend(loc='best')
fig.show()

Obviously, there is a lot going on there.

Yes, we still have the empirical AA Tau extinction curve, but all models will be normalized to some band, and usually the V band.  The better thing to do is fit the curve

First, find the observed Av:

In [7]:
Av=getMag('V',wavelength_dec02u,data_dec02u)-getMag('V',wavelength_2008u,data_2008u)
Av

NameError: name 'getMag' is not defined

**Option A:** Fit a line to $E(\lambda-V) = A_\lambda - A_V$

In [ ]:
fita,fitb=np.polyfit(1/notel_wav,notel_ext-Av,1)
fita,fitb,fita/fitb


So, $E(\lambda-V)=A_\lambda-A_V=1.54\lambda^{-1} - 2.72$.

This isn't internally consistent, however, because Av (2.03) should equal the intercept (2.72), and it doesn't.

What it implies is a NIR extinction curve of $A_\lambda/A_V = 0.56\lambda^{-1}$ normalized to the visible... as far as that goes.

**Option B:** Fit the offset power law to $E(\lambda-V)$

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def offpowlaw(x, a, b, c):
    return a * (x**b) + c

In [ ]:
cfit2,cerr2=curve_fit(offpowlaw,1/notel_wav,notel_ext-Av)
cfit2,cerr2,cfit2[0]/cfit2[2]

This says that $E(\lambda-V) = 1.95 \lambda^{-0.71} - 3.16 = 3.16 \left(0.62\lambda^{-0.71}-1\right)$, which maybe makes a some sense, but still the intercept is not equal to what we actually measure (and use to calculate the input to the fit!) for $A_V$.

**Option C:** Try fitting the FM07 and FM09 curve (which fits from the J band to the I band) to the $A_\lambda$ curve.

In [ ]:
def FM07(x, a, b):
    # x is wavelength, a is Av, b is Rv
    return a*(0.63 - 0.84/b)*x**-1.84

In [ ]:
cfit3,cerr3=curve_fit(FM07,notel_wav,notel_ext)
cfit3,cerr3

Obviously ridiculous. It didn't even remotely converge.  Why?

In [ ]:
a = np.linspace(0,10,100)
b = np.linspace(1,8,100)
x,y = np.meshgrid(a, b)
z = FM07(1,x,y) # initialize, and ext at 1 micron

for i in range(100):
    for j in range(100):
        z[i,j]=np.sqrt(np.sum((notel_ext-FM07(notel_wav,a[i],b[j]))**2.0))



In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))
ax1.pcolor(x,y,z,cmap='RdBu')
#ax1.colorbar()
ax1.contour(x,y,z,20,colors='k')
ax1.set_xlabel('Av')
ax1.set_ylabel('Rv')
ax1.set_title('AA Tau: FM07 NIR residual sum')
ax2.pcolor(x,y, FM07(1,x,y),cmap='RdBu')
#ax1.colorbar()
ax2.contour(x,y, FM07(1,x,y),20,colors='k')
ax2.set_xlabel('Av')
ax2.set_ylabel('Rv')
ax2.set_title('FM07 extinction at 1 micron')
plt.show()
plt.show()

In [ ]:
def FM09(x, a, b, c):
    # x is wavelength, A is Av, b is Rv, c is alpha
    return a*(0.349/b + 2.087)/(1 + (x/0.507)**c)

In [8]:
cfit4,cerr4=curve_fit(FM09,notel_wav,notel_ext)
cfit4,cerr4

NameError: name 'curve_fit' is not defined

Not a converged fit, except alpha?  Here's why:

In [ ]:
plt.plot(1/notel_wav,notel_ext,'black',alpha=0.5)
for alpha in [1,1.5,2,2.5,3]:
    plt.plot(1/notel_wav,FM09(notel_wav,2.0,3.0,alpha))

The amount of extinction of strongly dependent on alpha.  Strange feature of the FM09 function. 

In [ ]:
a = np.linspace(-5,10,100) # Av
b = np.linspace(1,8,100) # Rv
c = np.linspace(0,3,100) # alpha
aa, bb = np.meshgrid(a, b)
bb, cc = np.meshgrid(b, c)
cc, aa = np.meshgrid(c, a)

z_a2 =     FM09(1, 2.0, bb, cc) # initialize and 1 micron
z_r3 =     FM09(1, aa, 3.0, cc) # initialize and 1 micron
z_alpha25= FM09(1, aa, bb, 2.5) # initialize and 1 micron


for i in range(100):
    for j in range(100):
        z_a2[i,j]=np.sqrt(np.sum((notel_ext-FM09(notel_wav,2.0,b[i],c[j]))**2.0))
        z_r3[i,j]=np.sqrt(np.sum((notel_ext-FM09(notel_wav,a[i],3.0,c[j]))**2.0))
        z_alpha25[i,j]=np.sqrt(np.sum((notel_ext-FM09(notel_wav,a[i],b[j],2.5))**2.0))



**Option D:**  Fit just the power law part with a free exponent to $A_\lambda$. 

In [9]:
def powlaw(x, a, b):
    return a * (x**b)

In [10]:
cfit,cerr=curve_fit(powlaw,1/notel_wav,notel_ext)
cfit,cerr

NameError: name 'curve_fit' is not defined

This says that $A_\lambda=0.85 \lambda^{-2.55}$, which makes little sense as an extinction law.  Note that we removed $A_V$ as a free parameter.

**This is the way to go!  No muss, no fuss.**

In [11]:
cfit[0]/Av

NameError: name 'cfit' is not defined

With the $A_V$ that we measure, $A_\lambda/A_V=0.42\lambda^{-2.55}$.

In [ ]:
np.sqrt(np.mean(np.square(notel_ext-powlaw(1/notel_wav,cfit[0],cfit[1]))))

In [ ]:
np.square(2)

So we plot them:

In [ ]:
fig,(ax1,ax2)=plt.subplots(2,1,figsize=(10,10))


ax1.plot(1./allwav_dec02,ext_dec02,'k',alpha=0.5)
# ax1.plot(1/notel_wav,notel_ext,'black',alpha=0.5)
ax1.plot(1/notel_wav,cfit[0]/notel_wav**cfit[1],'blue',label='pow law fit to ext',linewidth=2)
ax1.set_xlabel('$1/\lambda$ [1/micron]')
ax1.set_ylabel('$A_\lambda$')
ax1.set_title('AA Tau Extinction Event')
ax1.set_ylim(-0.5,1.5)
ax1.set_xlim(0.3,1.3)
ax1.legend(loc='best')

ax2.plot(1./allwav_dec02,ext_dec02-Av,'k',alpha=0.5)
# ax2.plot(1/notel_wav,notel_ext-Av,'black',alpha=0.5)
ax2.plot(1/notel_wav,fita/notel_wav+fitb,'green',label='linear fit',linewidth=2)
ax2.plot(1/notel_wav,cfit2[0]/notel_wav**cfit2[1]+cfit2[2],'orange',label='offset pow law fit',linewidth=2)
ax2.set_xlabel('$1/\lambda$ [1/micron]')
ax2.set_ylabel('$E(\lambda - V)  =  A_\lambda - A_V$ (observed)')
ax2.set_ylim(-0.5-Av,1.5-Av)
ax2.set_xlim(0.3,1.3)
ax2.legend(loc='best')

fig.show()

**Repeat for Dec 12:**

In [ ]:
Av12=getMag('V',wavelength_dec12u,data_dec12u)-getMag('V',wavelength_2008u,data_2008u)
Av12

In [ ]:
IRwav_dec12clip=IRwavelength_dec12[IRwavelength_dec12<2.47]
IRdat_dec12clip=IRdata_dec12[IRwavelength_dec12<2.47]

IRext_dec12clip=-2.5*np.log10(IRdat_dec12clip/interp2008(IRwav_dec12clip))

notel_ext12clip=IRext_dec12clip[IRext_dec12clip > -0.1]
notel_wav12clip=IRwav_dec12clip[IRext_dec12clip > -0.1]

In [12]:
cfit12_2,cerr12_2=curve_fit(offpowlaw,1/notel_wav12clip,notel_ext12clip-Av12)
cfit12_2,cerr12_2,cfit12_2[0]/cfit12_2[2]

NameError: name 'curve_fit' is not defined

This says that $E(\lambda-V) = 2.37 \lambda^{-0.67} - 3.53 =  3.53 \left(0.67\lambda^{-0.67}-1\right)$.

In [ ]:
cfit12,cerr12=curve_fit(powlaw,1/notel_wav12clip,notel_ext12clip)
cfit12,cerr12

This says that $A_\lambda=0.98 \lambda^{-2.54}$.

## Grey extinction? <a name="grey"></a>

Grey extinction does not help in the extinction event of AA Tau because there is almost no extinction at K, too little in fact to yield a good fit.

But, what if the AA Tau lost some grey extinction that was present before the extinction event?  This would be the equivalent of shifting the whole extinction curve upward by the amount of that grey extinction.  In other words, the star was brighter behind the now-missing grey extinction.

So we fit an offset power law not the the reddening, as before, but to the extinction.

In [ ]:
cfit10,cerr10=curve_fit(offpowlaw,1/notel_wav,notel_ext)
cfit10,cerr10

So, $A_\lambda = 1.96\lambda^{-0.71}$ and $A_{\rm grey}=1.12$.  Notice that this did nothing to help the shape of the curve.

In [ ]:
fig,ax1=plt.subplots(1,1,figsize=(10,5))

Agrey=0.25

y=IRext_dec02
x=IRwavelength_dec02

fill=1/np.array([0.3,.2,.1,.01])

ax1.plot(1/x,y+Agrey,'black',alpha=0.5)
ax1.plot(1/allwav_dec02,ext_dec02+Agrey,'black',alpha=0.5)
ax1.plot(np.append(1/x,1/fill),powlaw(np.append(x,fill),(Av+Agrey)*0.4,-1.8),label='MW90 with grey',linewidth=2)
ax1.plot(np.append(1/x,1/fill),FM07(np.append(x,fill),(Av+Agrey),8),label='FM07 + RV=8 with grey',linewidth=2)

ax1.set_xlabel('$\lambda^{-1}$ [$\mu$m$^{-1}$]')
ax1.set_ylabel('$A_\lambda$')
# ax1.title('test')
ax1.legend(loc='best')
ax1.set_title('AA Tau with grey extinction')
ax1.set_ylim(0,4)
ax1.set_xlim(0,2.5)
fig.show()

# Write data to files <a name="write"></a>

In [13]:
from astropy.table import QTable
from astropy.io import ascii

t = QTable([IRwavelength_2008u.astype('float32'), IRdata_2008u.astype('float32')], names=['wavelength', 'flux'])
ascii.write(t, format='ecsv', output='AATau_IR2008.txt', overwrite=True)   

t = QTable([wavelength_2008u.astype('float32'), data_2008u.astype('float32')], names=['wavelength', 'flux'])
ascii.write(t, format='ecsv', output='AATau_opt2008.txt', overwrite=True)   

t = QTable([IRwavelength_dec02u.astype('float32'), IRdata_dec02u.astype('float32')], names=['wavelength', 'flux'])
ascii.write(t, format='ecsv', output='AATau_IR2014dec02.txt', overwrite=True)   

t = QTable([wavelength_dec02u.astype('float32'), data_dec02u.astype('float32')], names=['wavelength', 'flux'])
ascii.write(t, format='ecsv', output='AATau_opt2014dec02.txt', overwrite=True)   

t = QTable([IRwavelength_dec12u.astype('float32'), IRdata_dec12u.astype('float32')], names=['wavelength', 'flux'])
ascii.write(t, format='ecsv', output='AATau_IR2014dec12.txt', overwrite=True)   

t = QTable([wavelength_dec12u.astype('float32'), data_dec12u.astype('float32')], names=['wavelength', 'flux'])
# t['wavelength'] = t['wavelength'].astype('float32')
ascii.write(t, format='ecsv', output='AATau_opt2014dec12.txt', overwrite=True)   


NameError: name 'IRwavelength_2008u' is not defined

# Figure 1: AAVSO <a name="figures"></a>

# Figure 2: Spectra, Dec 02

In [ ]:
fig,(ax1,ax2)=plt.subplots(2,1,figsize=(10,10),gridspec_kw = {'height_ratios':[3, 1]})

# fig.subplots_adjust(hspace=0)

# my little FMunred doesn't include UV, lambda < 0.411 micron

ax1.plot(allwav_2008,alldata_2008,'grey',label='2008 observed')
ax1.plot(allwav_2008[allwav_2008>0.411],alldata_2008[allwav_2008>0.411]*
        10.0**(-0.4*2.0*getFMext(allwav_2008[allwav_2008>0.411],3.0,'fmunred')/3.0),'blue',label="2008 with AV=2.0 RV=3")
ax1.plot(allwav_dec02,alldata_dec02,'red',label='2014 Dec 2 observed',alpha=0.7)
ax1.set_ylim(0,0.8E-13)
ax1.set_xlim(0.25,2.5)
ax1.set_xlabel('$\lambda$ [micron]')
ax1.set_ylabel('$F_\lambda$ (erg s$^{-1}$ cm$^{-2}$ $\AA^{-1}$)')
ax1.legend(loc='upper right')

ax2.plot([.25,2.5],[0,0],'k')
ax2.plot(allwav_dec02[allwav_dec02>0.411],
         interp2008(allwav_dec02[allwav_dec02>0.411])*10.0**(-0.4*2.0*getFMext(allwav_dec02[allwav_dec02>0.411],3.0,'fmunred')/3.0)
         -alldata_dec02[allwav_dec02>0.411],'r',alpha=0.7)
ax2.set_xlim(0.25,2.5)
ax2.set_ylim(-1E-14,1E-14)
ax2.set_xlabel('$\lambda$ [micron]')
ax2.set_ylabel('Model $-$ Observed')

plt.show()

# Figure 3: Spectra, Dec 12

In [ ]:
fig,(ax1,ax2)=plt.subplots(2,1,figsize=(10,10),gridspec_kw = {'height_ratios':[3, 1]})

# fig.subplots_adjust(hspace=0)

# my little FMunred doesn't include UV, lambda < 0.411 micron

ax1.plot(allwav_2008,alldata_2008,'grey',label='2008 observed')
ax1.plot(allwav_2008[allwav_2008>0.411],alldata_2008[allwav_2008>0.411]*
        10.0**(-0.4*2.0*getFMext(allwav_2008[allwav_2008>0.411],3.0,'fmunred')/3.0),'blue',label="2008 with AV=2.0 RV=3")
ax1.plot(allwav_dec12,alldata_dec12,'orange',label='2014 Dec 12 observed',alpha=0.8)
ax1.set_ylim(0,0.8E-13)
ax1.set_xlim(0.25,2.5)
ax1.set_xlabel('$\lambda$ [micron]')
ax1.set_ylabel('$F_\lambda$ (erg s$^{-1}$ cm$^{-2}$ $\AA^{-1}$)')
ax1.legend(loc='upper right')

ax2.plot([.25,2.5],[0,0],'k')
clip12=np.logical_and(allwav_dec12>0.411,allwav_dec12<2.47)
ax2.plot(allwav_dec12[clip12],interp2008(allwav_dec12[clip12])*
         10.0**(-0.4*2.0*getFMext(allwav_dec12[clip12],3.0,'fmunred')/3.0)
         -alldata_dec12[clip12],'orange',alpha=0.8)
ax2.set_xlim(0.25,2.5)
ax2.set_ylim(-1E-14,1E-14)
ax2.set_xlabel('$\lambda$ [micron]')
ax2.set_ylabel('Model $-$ Observed')

plt.show()

# Figure: Color-color/mag plots

In [ ]:
allwav_2008clip = allwav_2008[allwav_2008>0.411] * u.micron
alldata_2008clip = alldata_2008[allwav_2008>0.411] * u.erg / (u.s * u.cm * u.cm * u.AA)

allwav_dec02clip = allwav_dec02[allwav_dec02>0.411] * u.micron
alldata_dec02clip = alldata_dec02[allwav_dec02>0.411] * u.erg / (u.s * u.cm * u.cm * u.AA)

allwav_dec12clip = allwav_dec12[allwav_dec12>0.411] * u.micron
alldata_dec12clip= alldata_dec12[allwav_dec12>0.411]  * u.erg / (u.s * u.cm * u.cm * u.AA)



bands=['Ks','H','J','I','R','V','B']

obs1ext = np.zeros((6,alldata_2008clip.size))    
fobs1ext = np.zeros((6,alldata_2008clip.size))    

j=0
for A in [2.0,4.0]:
    for R in [2.0,3.0,5.0]:
        obs1ext[j,:] = A*getFMext(allwav_2008clip.value,R,'fmunred')/R
        fobs1ext[j,:] = alldata_2008clip * 10.**(-0.4*obs1ext[j,:])
        j+=1

fobs1ext = fobs1ext * u.erg / (u.s * u.cm * u.cm * u.AA)
        
mags1ext=np.zeros((6,7))

mags1=np.zeros(7)
mags2=np.zeros(7)
mags3=np.zeros(7)

for j in [0,1,2,3,4,5,6]:
    for i in range(6):
        mags1ext[i,j]=getMag(bands[j], allwav_2008clip, fobs1ext[i,:])
    mags1[j]=getMag(bands[j], allwav_2008clip, alldata_2008clip)
    mags2[j]=getMag(bands[j], allwav_dec02clip, alldata_dec02clip)
    mags3[j]=getMag(bands[j], allwav_dec12clip, alldata_dec12clip)


Print magnitudes for 2008, 2014Dec02, and 2014Dec14 in BVRIJHK order

In [ ]:
mags1[::-1],mags2[::-1],mags3[::-1]

Dec 14 is slightly fainter than Dec 02 in all but K

In [ ]:
mags3-mags2

Print magnitudes for 2008 with extinction:

In [ ]:
mags1ext

In [ ]:
fig,axes=plt.subplots(2,6,figsize=(18,6))


for j in [0,1,2,3,4,5]: # just data
    if j!=5:
        axes[1,5-j].plot(mags2[j+1]-mags2[j],mags2[j+2]-mags2[j+1],'o',color='red')
        
        axes[1,5-j].annotate("",xy=(mags2[j+1]-mags2[j],mags2[j+2]-mags2[j+1]), xycoords='data',
            xytext=(mags1[j+1]-mags1[j],mags1[j+2]-mags1[j+1]),textcoords='data',
            arrowprops=dict(color='red',headwidth=8,headlength=12,width=0.1))
        
        axes[1,5-j].plot(mags1[j+1]-mags1[j],mags1[j+2]-mags1[j+1],'o',color='black',label='2008')
        axes[1,5-j].set_xlabel(bands[j+1]+'$-$'+bands[j])
        axes[1,5-j].set_ylabel(bands[j+2]+'$-$'+bands[j+1])
                        
    
    axes[0,5-j].plot(mags1[j+1]-mags1[j],mags1[j+1],'o',color='black')
    axes[0,5-j].plot(mags2[j+1]-mags2[j],mags2[j+1],'o',color='red')
    axes[0,5-j].annotate("",xy=(mags2[j+1]-mags2[j],mags2[j+1]), xycoords='data',
            xytext=(mags1[j+1]-mags1[j],mags1[j+1]),textcoords='data',
            arrowprops=dict(color='red',headwidth=8,headlength=12,width=0.1))
    axes[0,5-j].set_xlabel(bands[j+1]+'$-$'+bands[j])
    axes[0,5-j].set_ylabel(bands[j+1])
    

for j in [0,1,2,3,4,5]: # extinction 
    if j!=5:
        axes[1,5-j].plot(mags1ext[0,j+1]-mags1ext[0,j],mags1ext[0,j+2]-mags1ext[0,j+1],'+',color='blue',label='2008, AV=2.0 RV=2')
        axes[1,5-j].plot(mags1ext[1,j+1]-mags1ext[1,j],mags1ext[1,j+2]-mags1ext[1,j+1],'o',color='blue',label='2008, AV=2.0 RV=3')
        axes[1,5-j].plot(mags1ext[2,j+1]-mags1ext[2,j],mags1ext[2,j+2]-mags1ext[2,j+1],'x',color='blue',label='2008, AV=2.0 RV=5')
        axes[1,5-j].plot(mags1ext[3,j+1]-mags1ext[3,j],mags1ext[3,j+2]-mags1ext[3,j+1],'+',color='green',label='2008, AV=4.0 RV=2')
        axes[1,5-j].plot(mags1ext[4,j+1]-mags1ext[4,j],mags1ext[4,j+2]-mags1ext[4,j+1],'o',color='green',label='2008, AV=4.0 RV=3')
        axes[1,5-j].plot(mags1ext[5,j+1]-mags1ext[5,j],mags1ext[5,j+2]-mags1ext[5,j+1],'x',color='green',label='2008, AV=4.0 RV=5')

    axes[0,5-j].plot(mags1ext[0,j+1]-mags1ext[0,j],mags1ext[0,j+1],'+',color='blue')
    axes[0,5-j].plot(mags1ext[1,j+1]-mags1ext[1,j],mags1ext[1,j+1],'o',color='blue')
    axes[0,5-j].plot(mags1ext[2,j+1]-mags1ext[2,j],mags1ext[2,j+1],'x',color='blue')
    axes[0,5-j].plot(mags1ext[3,j+1]-mags1ext[3,j],mags1ext[3,j+1],'+',color='green')
    axes[0,5-j].plot(mags1ext[4,j+1]-mags1ext[4,j],mags1ext[4,j+1],'o',color='green')
    axes[0,5-j].plot(mags1ext[5,j+1]-mags1ext[5,j],mags1ext[5,j+1],'x',color='green')
    

axes[1,1].plot(mags2[5]-mags2[4],mags2[6]-mags2[5],'o',color='red',label='2014') # just for the legend
axes[1,0].axis('off')
fig.subplots_adjust(wspace=0.8,hspace=0.3)
axes[1,1].legend(loc='center left', bbox_to_anchor=(-2, 0.5))
for j in [0,1,2,3,4,5]:
    axes[0,5-j].set_ylim(axes[0,5-j].get_ylim()[::-1])

plt.show()



# Figure: Color-color/mag plots subset

In [ ]:
fig,axes=plt.subplots(1,2,figsize=(6,3))

j=5
axes[0].plot(mags1[j+1]-mags1[j],mags1[j],'o',color='black')
axes[0].plot(mags2[j+1]-mags2[j],mags2[j],'o',color='red')
axes[0].annotate("",xy=(mags2[j+1]-mags2[j],mags2[j]), xycoords='data',
    xytext=(mags1[j+1]-mags1[j],mags1[j]),textcoords='data',
    arrowprops=dict(color='red',headwidth=8,headlength=12,width=0.1))
axes[0].set_xlabel(bands[j+1]+'$-$'+bands[j])
axes[0].set_ylabel(bands[j])

axes[0].plot(mags1ext[0,j+1]-mags1ext[0,j],mags1ext[0,j],'+',color='blue')
axes[0].plot(mags1ext[1,j+1]-mags1ext[1,j],mags1ext[1,j],'o',color='blue')
axes[0].plot(mags1ext[2,j+1]-mags1ext[2,j],mags1ext[2,j],'x',color='blue')
axes[0].plot(mags1ext[3,j+1]-mags1ext[3,j],mags1ext[3,j],'+',color='green')
axes[0].plot(mags1ext[4,j+1]-mags1ext[4,j],mags1ext[4,j],'o',color='green')
axes[0].plot(mags1ext[5,j+1]-mags1ext[5,j],mags1ext[5,j],'x',color='green')


j=0     
axes[1].annotate("",xy=(mags2[j+1]-mags2[j],mags2[j+2]-mags2[j+1]), xycoords='data',
    xytext=(mags1[j+1]-mags1[j],mags1[j+2]-mags1[j+1]),textcoords='data',
    arrowprops=dict(color='red',headwidth=8,headlength=12,width=0.1))
axes[1].plot(mags1[j+1]-mags1[j],mags1[j+2]-mags1[j+1],'o',color='black',label='2008')
axes[1].set_xlabel(bands[j+1]+'$-$'+bands[j])
axes[1].set_ylabel(bands[j+2]+'$-$'+bands[j+1])

axes[1].plot(mags1ext[0,j+1]-mags1ext[0,j],mags1ext[0,j+2]-mags1ext[0,j+1],'+',color='blue',label='2008, AV=2.0 RV=2')
axes[1].plot(mags1ext[1,j+1]-mags1ext[1,j],mags1ext[1,j+2]-mags1ext[1,j+1],'o',color='blue',label='2008, AV=2.0 RV=3')
axes[1].plot(mags1ext[2,j+1]-mags1ext[2,j],mags1ext[2,j+2]-mags1ext[2,j+1],'x',color='blue',label='2008, AV=2.0 RV=5')
axes[1].plot(mags1ext[3,j+1]-mags1ext[3,j],mags1ext[3,j+2]-mags1ext[3,j+1],'+',color='green',label='2008, AV=4.0 RV=2')
axes[1].plot(mags1ext[4,j+1]-mags1ext[4,j],mags1ext[4,j+2]-mags1ext[4,j+1],'o',color='green',label='2008, AV=4.0 RV=3')
axes[1].plot(mags1ext[5,j+1]-mags1ext[5,j],mags1ext[5,j+2]-mags1ext[5,j+1],'x',color='green',label='2008, AV=4.0 RV=5')

axes[1].plot(mags2[j+1]-mags2[j],mags2[j+2]-mags2[j+1],'o',color='red',label='2014')  


fig.subplots_adjust(wspace=0.4)
axes[1].legend(loc='center right', bbox_to_anchor=(2.15, 0.5))


axes[0].set_ylim(axes[0].get_ylim()[::-1])


plt.show()

# Figure: Extinction and reddening

In [ ]:
fig,axs=plt.subplots(4,2,sharex='col',sharey='row',figsize=(15,18),gridspec_kw = {'height_ratios':[3, 2, 2, 3]})

# fig.subplots_adjust(hspace=0)

# my little FMunred doesn't include UV, lambda < 0.411 micron

axs[0,0].plot(allwav_2008clip,alldata_2008clip,'k',alpha=0.4,label='2008 observed')
axs[0,0].plot(allwav_dec02clip,alldata_dec02clip,'red',label='02dec2014 observed')
axs[0,1].plot(1/allwav_2008clip,alldata_2008clip,'k',alpha=0.4)
axs[0,1].plot(1/allwav_dec02clip,alldata_dec02clip,'red')

axs[0,0].plot(allwav_2008clip,fobs1ext[0,:],'b',label='2008 with AV=2.0 RV=2',linestyle='dotted',alpha=0.5)
axs[0,1].plot(1/allwav_2008clip,fobs1ext[0,:],'b',linestyle='dotted',alpha=0.5)
axs[0,0].plot(allwav_2008clip,fobs1ext[1,:],'b',label='2008 with AV=2.0 RV=3',linestyle='solid',alpha=0.5)
axs[0,1].plot(1/allwav_2008clip,fobs1ext[1,:],'b',linestyle='solid',alpha=0.5)
axs[0,0].plot(allwav_2008clip,fobs1ext[2,:],'b',label='2008 with AV=2.0 RV=5',linestyle='dashed',alpha=0.5)
axs[0,1].plot(1/allwav_2008clip,fobs1ext[2,:],'b',linestyle='dashed',alpha=0.5)

axs[0,0].plot(allwav_2008clip,fobs1ext[3,:],'g',label='2008 with AV=4.0 RV=2',linestyle='dotted',alpha=0.5)
axs[0,1].plot(1/allwav_2008clip,fobs1ext[3,:],'g',linestyle='dotted',alpha=0.5)
axs[0,0].plot(allwav_2008clip,fobs1ext[4,:],'g',label='2008 with AV=4.0 RV=3',linestyle='solid',alpha=0.5)
axs[0,1].plot(1/allwav_2008clip,fobs1ext[4,:],'g',linestyle='solid',alpha=0.5)
axs[0,0].plot(allwav_2008clip,fobs1ext[5,:],'g',label='2008 with AV=4.0 RV=5',linestyle='dashed',alpha=0.5)
axs[0,1].plot(1/allwav_2008clip,fobs1ext[5,:],'g',linestyle='dashed',alpha=0.5)


axs[0,0].set_xlim(0.25,2.75)
axs[0,0].set_ylim(0,0.7E-13)
axs[0,0].set_ylabel('$F_\lambda$ (erg s$^{-1}$ cm$^{-2}$ $\AA^{-1}$)')
axs[0,0].legend(loc='best')
axs[0,1].set_xlim(0,2.5)
axs[0,1].set_ylim(0,0.7E-13)


axs[1,0].set_ylabel('$A_\lambda$')
axs[1,0].set_ylim(-0.5,5.1)
axs[1,0].plot(allwav_dec02[allwav_dec02>0.411],ext_dec02[allwav_dec02>0.411],'k',alpha=1,label='observed extinction')
axs[1,1].plot(1./allwav_dec02[allwav_dec02>0.411],ext_dec02[allwav_dec02>0.411],'k',alpha=1)
for j,line,legend in list(zip(range(3),('dotted','solid','dashed'),('2','3','5'))):
        axs[1,0].plot(allwav_2008clip,obs1ext[j,:],'b',linestyle=line,label='AV=2.0 RV='+legend+' law')
        axs[1,1].plot(1/allwav_2008clip,obs1ext[j,:],'b',linestyle=line)
        axs[1,0].plot(allwav_2008clip,obs1ext[j+3,:],'g',linestyle=line,label='AV=4.0 RV='+legend+' law')
        axs[1,1].plot(1/allwav_2008clip,obs1ext[j+3,:],'g',linestyle=line)
axs[1,0].legend(loc='best')
        
axs[1,0].plot(allwav_dec02[allwav_dec02>0.411],ext_dec02[allwav_dec02>0.411],'k',alpha=1)
axs[1,1].plot(1./allwav_dec02[allwav_dec02>0.411],ext_dec02[allwav_dec02>0.411],'k',alpha=1)
for j,line in list(zip(range(3),('dotted','solid','dashed'))):
        axs[1,0].plot(allwav_2008clip,obs1ext[j,:],'b',linestyle=line)
        axs[1,1].plot(1/allwav_2008clip,obs1ext[j,:],'b',linestyle=line)

axs[2,0].set_ylabel('$E(\lambda-V)$')      
axs[2,0].plot(allwav_dec02[allwav_dec02>0.411],ext_dec02[allwav_dec02>0.411]-2.0,'k',alpha=1)
axs[2,1].plot(1./allwav_dec02[allwav_dec02>0.411],ext_dec02[allwav_dec02>0.411]-2.0,'k',alpha=1)
for j,line, in list(zip(range(3),('dotted','solid','dashed'))):
        axs[2,0].plot(allwav_2008clip,obs1ext[j,:]-2.0,'b',linestyle=line)
        axs[2,1].plot(1/allwav_2008clip,obs1ext[j,:]-2.0,'b',linestyle=line)
        axs[2,0].plot(allwav_2008clip,obs1ext[j+3,:]-4.0,'g',linestyle=line)
        axs[2,1].plot(1/allwav_2008clip,obs1ext[j+3,:]-4.0,'g',linestyle=line)
axs[2,0].set_ylim(-4.1,2.1)

AH=mags2[1]-mags1[1]
AK=mags2[0]-mags1[0]
AHext=mags1ext[:,1]-mags1[1]
AKext=mags1ext[:,0]-mags1[0]

axs[3,0].set_ylabel('$E(\lambda-K)$')      
axs[3,0].plot(allwav_dec02[allwav_dec02>0.411],(ext_dec02[allwav_dec02>0.411]-AK),'k',alpha=0.7,label='2008 -> 2014')
axs[3,1].plot(1./allwav_dec02[allwav_dec02>0.411],(ext_dec02[allwav_dec02>0.411]-AK),'k',alpha=0.7)
for j,line, in list(zip(range(3),('dotted','solid','dashed'))):
        axs[3,0].plot(allwav_2008clip,(obs1ext[j,:]-AKext[j]),'b',linestyle=line)
        axs[3,1].plot(1/allwav_2008clip,(obs1ext[j,:]-AKext[j]),'b',linestyle=line)
        axs[3,0].plot(allwav_2008clip,(obs1ext[j+3,:]-AKext[j+3]),'g',linestyle=line)
        axs[3,1].plot(1/allwav_2008clip,(obs1ext[j+3,:]-AKext[j+3]),'g',linestyle=line)
axs[3,0].set_ylim(-0.5,5.1)

axs[3,0].set_xlabel('$\lambda$ [micron]')
axs[3,1].set_xlabel('$\lambda^{-1}$ [micron$^{-1}$]')


fig.subplots_adjust(hspace=0.05)
fig.subplots_adjust(wspace=0.03)

plt.show()

# Figure: Just infrared, fits to extinction and reddening

In [ ]:
Av

In [ ]:
fig,(ax1,ax2)=plt.subplots(2,1,sharex='col',figsize=(6,8))

ax1.plot(1./allwav_dec02,ext_dec02,'k',alpha=0.3)
# ax1.plot(1/notel_wav,notel_ext,'black',alpha=0.5)
ax1.plot(1/notel_wav,cfit[0]/notel_wav**cfit[1],'k',label='pow law fit',linewidth=2)
ax1.set_ylabel('$A_\lambda$')
ax1.set_ylim(-0.5,1.5)
ax1.set_xlim(0.3,1.3)
ax1.legend(loc='best')

ax2.plot(1./allwav_dec02,ext_dec02-Av,'k',alpha=0.3)
# ax2.plot(1/notel_wav,notel_ext-Av,'black',alpha=0.5)
ax2.plot(1/notel_wav,fita/notel_wav+fitb,'k',label='linear fit',linewidth=2,linestyle='dotted')
ax2.plot(1/notel_wav,cfit2[0]/notel_wav**cfit2[1]+cfit2[2],'k',label='offset power law fit',linewidth=2)
ax2.set_xlabel('$\lambda^{-1}$ [micron$^{-1}$]')
ax2.set_ylabel('$E(\lambda - V)  =  A_\lambda - A_V$ (observed)')
ax2.set_ylim(-0.5-Av,1.5-Av)
ax2.set_xlim(0.3,1.3)
ax2.legend(loc='best')


fig.subplots_adjust(hspace=0.05)
fig.subplots_adjust(wspace=0.03)

plt.show()



# Figure: Just infrared, extinction and reddening with models

In [ ]:
from dust_extinction import dust_extinction as de

In [ ]:
CCM89 = de.CCM89(Rv=3.0)
O94 = de.O94(Rv=3.0)
F99 = de.F99(Rv=3.0)
F04 = de.F04(Rv=3.0)
GCC09_MWAvg = de.GCC09_MWAvg()
G16 = de.G16(RvA=3.0, fA=1.0)

wavelengths=allwav_dec02*u.micron

fig,axs=plt.subplots(3,2,sharex='col',sharey='row',figsize=(8,12))

def plotlaws(i):
    axs[i,0].plot(allwav_dec02,ext_dec02,'k',alpha=0.3)
    axs[i,0].plot(wavelengths, -2.5*np.log10(CCM89.extinguish(wavelengths, Av=2)),label='CCM89, Av=2')
    axs[i,0].plot(wavelengths, -2.5*np.log10(O94.extinguish(wavelengths, Av=2)),label='O94, Av=2')
    axs[i,0].plot(wavelengths, -2.5*np.log10(F99.extinguish(wavelengths, Av=2)),label='F99, Av=2')
    axs[i,0].plot(wavelengths, -2.5*np.log10(F04.extinguish(wavelengths, Av=2)),label='F04, Av=2')
    axs[i,0].plot(wavelengths, -2.5*np.log10(G16.extinguish(wavelengths, Av=2)),label='G16, Av=2')

    axs[i,1].plot(1./allwav_dec02,ext_dec02,'k',alpha=0.3)
    axs[i,1].plot(1./wavelengths, -2.5*np.log10(CCM89.extinguish(wavelengths, Av=2)),label='CCM89, Av=2')
    axs[i,1].plot(1./wavelengths, -2.5*np.log10(O94.extinguish(wavelengths, Av=2)),label='O94, Av=2')
    axs[i,1].plot(1./wavelengths, -2.5*np.log10(F99.extinguish(wavelengths, Av=2)),label='F99, Av=2')
    axs[i,1].plot(1./wavelengths, -2.5*np.log10(F04.extinguish(wavelengths, Av=2)),label='F04, Av=2')
    axs[i,1].plot(1./wavelengths, -2.5*np.log10(G16.extinguish(wavelengths, Av=2)),label='G16, Av=2')
    

plotlaws(1)
axs[1,0].plot(wavelengths, -2.5*np.log10(GCC09_MWAvg.extinguish(wavelengths, Av=2)),label='GCC09, Av=2')
axs[1,1].plot(1./wavelengths, -2.5*np.log10(GCC09_MWAvg.extinguish(wavelengths, Av=2)),label='GCC09, Av=2')
        
CCM89 = de.CCM89(Rv=2.0)
O94 = de.O94(Rv=2.0)
F99 = de.F99(Rv=2.0)
F04 = de.F04(Rv=2.0)
G16 = de.G16(RvA=2.0, fA=1.0)

plotlaws(0)

CCM89 = de.CCM89(Rv=5.0)
O94 = de.O94(Rv=5.0)
F99 = de.F99(Rv=5.0)
F04 = de.F04(Rv=5.0)
G16 = de.G16(RvA=5.0, fA=1.0)

plotlaws(2)

axs[0,0].set_ylabel('$A_\lambda$')
axs[1,0].set_ylabel('$A_\lambda$')
axs[2,0].set_ylabel('$A_\lambda$')
axs[0,0].set_ylim(-0.2,4)
axs[1,0].set_ylim(-0.2,4)
axs[2,0].set_ylim(-0.2,4)
axs[1,0].legend(loc='best')

axs[2,0].set_xlabel('$\lambda$ [micron]')
axs[2,1].set_xlabel('$\lambda^{-1}$ [micron$^{-1}$]')

axs[0,1].text(.5,3.5,'Rv=2',fontsize='13')
axs[1,1].text(.5,3.5,'Rv=3',fontsize='13')
axs[2,1].text(.5,3.5,'Rv=5',fontsize='13')

fig.subplots_adjust(hspace=0.05)
fig.subplots_adjust(wspace=0.03)


plt.show()